In [1]:
from Lib.myModules.my_ML_Modules_1 import *

In [3]:
data_dir = "../data"
infantNum = 3
ECG_dataset = f"{data_dir}/infant{infantNum}_ecg"
RESP_dataset = f"{data_dir}/infant{infantNum}_resp"
# Read all the data
signal_ecg_0 = wfdb.rdsamp(f"{data_dir}/infant{infantNum}_ecg")
signal_resp_0 = wfdb.rdsamp(f"{data_dir}/infant{infantNum}_resp")
print(f'ECG DATA: {signal_ecg_0[1]}')
print(F'RESP DATA: {signal_resp_0[1]}')
totalDuration_ECG = num2Time(num=signal_ecg_0[1]['sig_len'], fs=signal_ecg_0[1]['fs'])
print(f'{totalDuration_ECG} sec, {secs2minutes(totalDuration_ECG)} minutes, {secs2hours(totalDuration_ECG)} hours')
totalDuration_RESP = num2Time(num=signal_resp_0[1]['sig_len'], fs=signal_resp_0[1]['fs'])
print(f'{totalDuration_RESP} sec, {secs2minutes(totalDuration_RESP)} minutes, {secs2hours(totalDuration_RESP)} hours')
# Start and end time defined
start_time_secs = hours2secs(1)
end_time_secs = hours2secs(1.1)
ECG_startNum = time2Num(start_time_secs, signal_ecg_0[1]['fs'])
ECG_endNum = time2Num(end_time_secs, signal_ecg_0[1]['fs'])
print(f'start: {ECG_startNum}, end: {ECG_endNum}')
RESP_startNum = time2Num(start_time_secs, signal_resp_0[1]['fs'])
RESP_endNum = time2Num(end_time_secs, signal_resp_0[1]['fs'])
print(f'start: {RESP_startNum}, end: {RESP_endNum}')
signal_ECG, signal_ECG_startTime_secs, signal_ECG_endTime_secs = read_Data(ECG_dataset, startNum=ECG_startNum, endNum=ECG_endNum)
signal_RESP, signal_RESP_startTime_secs, signal_RESP_endTime_secs = read_Data(RESP_dataset, startNum=RESP_startNum, endNum=RESP_endNum)
# print(signal_ECG[0].shape)
signal_ECG_1 = iqr_smooth(signal=signal_ECG[0],hiPerc=90,loPerc=10, cutoff_factor=1)
# print(signal_RESP[0].shape)
signal_RESP_1 = iqr_smooth(signal=signal_RESP[0],hiPerc=90,loPerc=10, cutoff_factor=1)
signal_ECG_2 = norm_signal(signal=signal_ECG_1)
signal_RESP_2 = norm_signal(signal=signal_RESP_1)
hrs_0 = cal_heartrate(signal=signal_ECG_2, fs=signal_ECG[1]['fs'])
rrs_0 = cal_resprate(signal=signal_RESP_2, fs=signal_RESP[1]['fs'])
if (rrs_0.shape[0] < hrs_0.shape[0]):
    hrs_1 = data_resample(data_fixNan(hrs_0), rrs_0.shape[0])
    rrs_1 = data_fixNan(rrs_0)
else:
    rrs_1 = data_resample(data_fixNan(rrs_0), hrs_0.shape[0])
    hrs_1 = data_fixNan(hrs_0)
correlationTest(rrs_1[:],hrs_1[:], plot=False)
print("---------------------------------------------")
print("--- Linear Regression ---")
lr_model_1 = train_lr_model(X=rrs_1, y=hrs_1)
print("---------------------------------------------")
print("--- Polynomial Regression ---")
pr_model_1 = train_pr_model(X=rrs_1, y=hrs_1, degree=6)
print("---------------------------------------------")
print("--- SVR Radius Basis Function---")
svr_rbf_1 = train_svr_rbf_model(X=rrs_1, y=hrs_1, param_C=100, param_gamma=0.1, seed=42)
print("--- SVR Linear ---")
svr_lin_1 = train_svr_lin_model(X=rrs_1, y=hrs_1, param_C=1000, param_gamma='auto', seed=42)
print("--- SVR Polynomial ---")
svr_poly_1 = train_svr_poly_model(X=rrs_1, y=hrs_1, param_C=1000, param_gamma='auto', param_degree=6, seed=42)

ECG DATA: {'fs': 500, 'sig_len': 78684614, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['mV'], 'sig_name': ['II'], 'comments': []}
RESP DATA: {'fs': 50, 'sig_len': 7868296, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['NU'], 'sig_name': ['RESP'], 'comments': []}
157369.228 sec, 2622.8204666666666 minutes, 43.71367444444444 hours
157365.92 sec, 2622.7653333333337 minutes, 43.71275555555556 hours
start: 1800000, end: 1980000
start: 180000, end: 198000
90th percentile: 0.16400837520909167, 10th percentile: -0.23755355254763474, IQR: 0.4015619277567264
Identified outliers: 9621
Data Shape: (180000,)
90th percentile: 24.07121214669968, 10th percentile: 20.537240474173423, IQR: 3.533971672526256
Identified outliers: 83
Data Shape: (18000,)
[[141.78716262  19.45454482]
 [ 19.45454482 249.79800093]]
Pearsons correlation: 0.103
Spearmans correlation: 0.074
---------------------------------------------
--- Linear Regression ---
Coefficient of determination: 0.009639